# Proyecto final: Cancelación de clientes en telecomunicaciones

Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. El equipo de marketing de Interconnect ha recopilado algunos de los datos personales de sus clientes, incluyendo información sobre sus planes y contratos.

Explicación de los datasets:

<b>contract.csv</b>

<ul>
<li><b>customerID:</b> ID del cliente.</li>
<li><b>BeginDate:</b> Fecha en la que contrató el servicio.</li>
<li><b>EndDate:</b> Fecha en la que canceló el servicio si es que canceló, si no es 'No'.</li>
<li><b>Type:</b> Si paga mensual, anualmente o cada dos años.</li>
<li><b>PaperlessBilling:</b> Si el recibo llega físico (Yes/NO).</li>
<li><b>PaymentMethod:</b> Método de pago.</li>
<li><b>MontlyCharges:</b> Cobro mensual.</li>
<li><b>TotalCharges:</b> Total que se le ha cobrado al cliente.</li>
</ul>

<b>internet.csv</b>

<ul>
<li><b>customerID:</b> ID del cliente.</li>
<li><b>InternetService:</b> Tiene este servicio (Yes/NO).</li>
<li><b>OnlineSecurity:</b> Tiene este servicio (Yes/NO). </li>
<li><b>OnlineBackup:</b> Tiene este servicio (Yes/NO).</li>
<li><b>DeviceProtection:</b> Tiene este servicio (Yes/NO).</li>
<li><b>TechSupport:</b> Tiene este servicio (Yes/NO).</li>
<li><b>StreamingTV:</b> Tiene este servicio (Yes/NO).</li>
<li><b>StreamingMovies:</b> Tiene este servicio (Yes/NO).</li>
</ul>

<b>personal.csv</b>

<ul>
<li><b>customerID:</b> ID del cliente.</li>
<li><b>gender:</b> Género.</li>
<li><b>SeniorCitizen:</b> Si el cliente es adulto mayor.</li>
<li><b>Partner:</b> Si el cliente vive con una pareja .</li>
<li><b>Dependents:</b> Si el cliente tiene personas a su cargo como hijos o adultos mayores.</li>
</ul>

<b>phone.csv</b>

<ul>
<li><b>customerID:</b> ID del cliente.</li>
<li><b>MultipleLines:</b> Tiene más de una línea (yes/no).</li>
</ul>


## Plan de trabajo

¿Qué factores influyen más a que la gente cancele el servicio?

¿Podemos crear un modelo que pueda predecir precisamente los clientes que cancelan?

¿Qué tanto influye el sercicio (internet/teléfono) que tienen contratado?

Para resolver la tarea, propongo el siguiente plan de trabajo:

1. Cargar los datasets y unirlos en uno solo.

2. Rellenar los datos ausentes para que reflejen la asuencia de ciertos clientes en los planes que no tienen contratados.

3. Convertir la columna objetivo de end_date a positivo/negativo (0/1)

4. Transformar los datos del resto de columnas a numéricos para que sean más fáciles de trabjar para modelos de ML.

5. Probar diferentes modelos de clasificación y escoger el que mejor métricas tenga, sugiero basarnos en f1 score en vez de AUC/ROC.

6. Ya que tengamos el mejor modelo, considero que podemos analizar los pesos de cada variable para ver cuál influye más en la tasa de cancelación.

## Carga de datos e importación de librerías

In [1]:
import pandas as pd
import numpy as np
import re
from boruta import BorutaPy
from sklearn.metrics import f1_score, roc_auc_score, recall_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score


In [2]:
contract = pd.read_csv('dataset/contract.csv')
internet = pd.read_csv('dataset/internet.csv')
personal = pd.read_csv('dataset/personal.csv')
phone = pd.read_csv('dataset/phone.csv')

## EDA

Primero vamos a ver cada dataframe y ver qué información nos dan

In [3]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


In [4]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [5]:
personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [6]:
phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


De entrada, vemos que todos los dataframes tienen customerID, lo que significa que podemos unir todos los archivos en un sólo dataframe para trabajar con él. También vemos que no hay valores ausentes en ninguno de nuestros dataframes

In [7]:
contract.sample(5)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
6260,0458-HEUZG,2019-01-01,No,Two year,No,Mailed check,35.40,450.4
2583,6463-MVYRY,2015-05-01,No,Two year,No,Bank transfer (automatic),69.85,4003
5373,4043-MKDTV,2014-03-01,No,Two year,Yes,Electronic check,105.25,7291.75
6898,1956-YIFGE,2018-04-01,No,One year,Yes,Mailed check,100.05,2090.25


In [8]:
internet.sample(5)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2155,0815-MFZGM,Fiber optic,No,No,Yes,Yes,Yes,Yes
2088,4328-VUFWD,DSL,No,No,No,Yes,No,Yes
888,3312-UUMZW,Fiber optic,No,Yes,No,No,Yes,Yes
4191,8992-OBVDG,DSL,No,No,Yes,No,No,Yes
5124,9317-WZPGV,Fiber optic,No,No,No,No,No,Yes


In [9]:
personal.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents
6119,0080-EMYVY,Female,0,No,No
5351,5649-VUKMC,Female,0,No,No
3427,9919-KNPOO,Female,0,Yes,No
1173,0107-WESLM,Male,0,No,No
403,3067-SVMTC,Female,0,Yes,No


In [10]:
phone.sample(5)

,customerID,MultipleLines
4575,1561-BWHIN,No
4949,2207-OBZNX,No
825,3865-YIOTT,Yes
5721,0547-HURJB,No
448,7803-XOCCZ,No


In [11]:
df = (
    contract.merge(internet, on='customerID', how='outer')
            .merge(personal, on='customerID', how='outer')
            .merge(phone, on='customerID', how='outer')
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   InternetService   5517 non-null   object 
 9   OnlineSecurity    5517 non-null   object 
 10  OnlineBackup      5517 non-null   object 
 11  DeviceProtection  5517 non-null   object 
 12  TechSupport       5517 non-null   object 
 13  StreamingTV       5517 non-null   object 
 14  StreamingMovies   5517 non-null   object 
 15  gender            7043 non-null   object 
 16  SeniorCitizen     7043 non-null   int64  


In [12]:
df['customerID'].duplicated().sum()

0

Ahora que tenemos un sólo dataframe unido, sin customerID duplicado, cambiamos los nombres de las columnas para que tengan snake casing

In [13]:
def snake_case(s):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', s).lower()

df.columns = [snake_case(col) for col in df.columns]

df.rename(columns={'customer_i_d' : 'customer_id', 'streaming_t_v': 'streaming_tv', 'type' : 'payment_type'}, inplace=True)

print(df.columns)

Index(['customer_id', 'begin_date', 'end_date', 'payment_type',
       'paperless_billing', 'payment_method', 'monthly_charges',
       'total_charges', 'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines'],
      dtype='object')


Ahora tenemos que ver cómo manejar los datos ausentes, ya que son muchos como para descartarlos

In [14]:
df[df.isnull().any(axis=1)].sample(5)

,customer_id,begin_date,end_date,payment_type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines
7021,9962-BFPDU,2020-01-01,No,Month-to-month,No,Mailed check,20.05,20.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,0,Yes,Yes,No
5174,7277-KAMWT,2019-01-01,No,Two year,No,Credit card (automatic),20.00,268.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,0,No,No,No
854,1240-HCBOH,2014-07-01,No,Two year,Yes,Mailed check,26.10,1759.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,0,No,No,Yes
4845,6828-HMKWP,2019-02-01,No,Two year,No,Bank transfer (automatic),21.05,262.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,0,Yes,Yes,No
1563,2274-XUATA,2014-02-01,No,Two year,Yes,Bank transfer (automatic),63.10,4685.55,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Male,1,Yes,No,NaN


In [15]:
#Convertir la columna objetivo a positivos y negativos y cambiar el nombre
df['end_date'] = np.where(df['end_date'] == 'No', 0, 1)
df.rename(columns={'end_date': 'churn'}, inplace=True)

In [16]:
#cambiamos a formato de fecha y dejamos sólo el año
df['begin_date'] = pd.to_datetime(df['begin_date'])
df['begin_year'] = df['begin_date'].dt.year
df.drop(columns=['begin_date'], inplace=True)

Vemos también que la columna total_charges es de tipo object, lo que significa que hay un error en alguna fila que no permite que la columna sea tratada como float, vamos a ver cuál es la causa

In [17]:
is_numeric = df['total_charges'].astype(str).str.strip().str.replace('.', '', 1).str.isdigit()

errors = df[~is_numeric]
print(f"Filas con errores: {len(errors)}")
errors[['total_charges']]

Filas con errores: 11


,total_charges
945,
1731,
1906,
2025,
2176,
2250,
2855,
3052,
3118,
4054,


En efecto, hay 11 filas que causan un problema porque no tienen datos. Son bastante pocas así que pueden eliminar.

In [18]:
df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce')
df = df.dropna(subset=['total_charges'])

In [19]:
#cambiamos columnas binarias a valores numéricos, manteniendo los valores nulos
binary_columns = ['paperless_billing', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'multiple_lines', 'partner', 'dependents']
for col in binary_columns:
    df[col] = df[col].map({'Yes': 1, 'No': 0}).where(df[col].notna(), df[col])

In [20]:
df.sample()

,customer_id,churn,payment_type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,begin_year
6599,9378-FXTIZ,1,One year,1,Credit card (automatic),70.15,3715.65,DSL,0.0,0.0,0.0,0.0,1.0,1.0,Female,0,1,0,1.0,2015


In [21]:
print('Unique Type:' , df['payment_type'].unique())
print('Unique Payment Method:' , df['payment_method'].unique())
print('Unique internet_service:', df['internet_service'].unique())

Unique Type: ['One year' 'Month-to-month' 'Two year']
Unique Payment Method: ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Unique internet_service: ['DSL' 'Fiber optic' nan]


In [22]:
#tansformamos el resto de columnas categóricas a numéricas con mapeo
df['payment_type'] = df['payment_type'].map({'Month-to-month': 0, 'One year': 1, 'Two year': 2})
df['payment_method'] = df['payment_method'].map({
    'Electronic check': 0,
    'Mailed check': 1,
    'Bank transfer (automatic)': 2,
    'Credit card (automatic)': 3
})
df['internet_service'] = df['internet_service'].map({'DSL': 1, 'Fiber optic': 2}).where(df['internet_service'].notna(), df['internet_service']).fillna(0)
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})
df.sample()

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19260\1640476755.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['internet_service'] = df['internet_service'].map({'DSL': 1, 'Fiber optic': 2}).where(df['internet_service'].notna(), df['internet_service']).fillna(0)


,customer_id,churn,payment_type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,begin_year
1597,2324-EFHVG,0,2,1,2,104.4,6692.65,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0,0,0,0,1.0,2014


Los valores ausentes se deben a que el cliente no estaba registrado en los respectivos dataframes, por lo que considero adecuado llenar todos estos con un 'No', de acuerdo a la notación de cada columna. Aunque es probable que se deba a otro factor, como una base de datos incompleta, así que haremos la prueba con ambos tipos de caso.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7032 non-null   object 
 1   churn              7032 non-null   int32  
 2   payment_type       7032 non-null   int64  
 3   paperless_billing  7032 non-null   int64  
 4   payment_method     7032 non-null   int64  
 5   monthly_charges    7032 non-null   float64
 6   total_charges      7032 non-null   float64
 7   internet_service   7032 non-null   float64
 8   online_security    5512 non-null   object 
 9   online_backup      5512 non-null   object 
 10  device_protection  5512 non-null   object 
 11  tech_support       5512 non-null   object 
 12  streaming_tv       5512 non-null   object 
 13  streaming_movies   5512 non-null   object 
 14  gender             7032 non-null   int64  
 15  senior_citizen     7032 non-null   int64  
 16  partner            7032 non-n

In [24]:
#Creamos los dos dataframes
df_no = df.copy().fillna(0)


df_alt = df.copy()
df_alt[binary_columns] = df_alt[binary_columns].fillna(2)

C:\Users\Oliver\AppData\Local\Temp\ipykernel_19260\4100383662.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_no = df.copy().fillna(0)
C:\Users\Oliver\AppData\Local\Temp\ipykernel_19260\4100383662.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_alt[binary_columns] = df_alt[binary_columns].fillna(2)


In [25]:
df_no.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7032 non-null   object 
 1   churn              7032 non-null   int32  
 2   payment_type       7032 non-null   int64  
 3   paperless_billing  7032 non-null   int64  
 4   payment_method     7032 non-null   int64  
 5   monthly_charges    7032 non-null   float64
 6   total_charges      7032 non-null   float64
 7   internet_service   7032 non-null   float64
 8   online_security    7032 non-null   float64
 9   online_backup      7032 non-null   float64
 10  device_protection  7032 non-null   float64
 11  tech_support       7032 non-null   float64
 12  streaming_tv       7032 non-null   float64
 13  streaming_movies   7032 non-null   float64
 14  gender             7032 non-null   int64  
 15  senior_citizen     7032 non-null   int64  
 16  partner            7032 non-n

In [26]:
df_alt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7032 non-null   object 
 1   churn              7032 non-null   int32  
 2   payment_type       7032 non-null   int64  
 3   paperless_billing  7032 non-null   int64  
 4   payment_method     7032 non-null   int64  
 5   monthly_charges    7032 non-null   float64
 6   total_charges      7032 non-null   float64
 7   internet_service   7032 non-null   float64
 8   online_security    7032 non-null   float64
 9   online_backup      7032 non-null   float64
 10  device_protection  7032 non-null   float64
 11  tech_support       7032 non-null   float64
 12  streaming_tv       7032 non-null   float64
 13  streaming_movies   7032 non-null   float64
 14  gender             7032 non-null   int64  
 15  senior_citizen     7032 non-null   int64  
 16  partner            7032 non-n

## Selección de características

Vamos a probar diferentes modelos, pero primero haremos una prueba con el módulo Boruta para ver cuáles parámetros son significativos.

In [27]:
#boruta para df_no
X = df_no.drop(columns=['customer_id', 'churn'])
y = df_no['churn']

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    max_iter=100,
    random_state=42,
    verbose=1
)

boruta_selector.fit(X.values, y.values)
# Variables seleccionadas (True/False por cada feature)
selected_mask = boruta_selector.support_

# Nombres de las features seleccionadas
selected_features = X.columns[selected_mask]

print("Features seleccionadas por Boruta:")
print(selected_features)


Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	14
Features seleccionadas por Boruta:
Index(['payment_type', 'monthly_charges', 'total_charges', 'begin_year'], dtype='object')


In [28]:
#boruta para df_alt
X = df_alt.drop(columns=['customer_id', 'churn'])
y = df_alt['churn']

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    max_iter=100,
    random_state=42,
    verbose=1
)

boruta_selector.fit(X.values, y.values)
# Variables seleccionadas (True/False por cada feature)
selected_mask = boruta_selector.support_

# Nombres de las features seleccionadas
selected_features = X.columns[selected_mask]

print("Features seleccionadas por Boruta:")
print(selected_features.tolist())


Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100


BorutaPy finished running.

Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	14
Features seleccionadas por Boruta:
['payment_type', 'monthly_charges', 'total_charges', 'begin_year']


Para ambos dataframes nos dio el mismo resultado de las columnas importantes, podemos hacer la prueba para ver si en efecto el modelo es mejor con sólo esas features

In [29]:
def split(df, target='churn', test_size=0.25, random_state=42):
    X = df.drop(columns=['customer_id', target])
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

X_train_no, X_test_no, y_train_no, y_test_no = split(df_no)
X_train_alt, X_test_alt, y_train_alt, y_test_alt = split(df_alt)

In [30]:
#Dataframe rellenado con 0 con características de Boruta
#Como las columnas que se rellenaron con 0 no aparecen en la lista de Boruta, no es necesario hacerlo con df_alt
df_bor = df_no.copy()[selected_features.tolist() + ['churn']]
X = df_bor.drop(columns= 'churn')
y = df_bor['churn']
X_train_bor, X_test_bor, y_train_bor, y_test_bor = train_test_split(X, y, test_size=0.25, random_state=42)

In [31]:
def test_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators=100, random_state=314)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)
    f1 = f1_score(y_test, predictions)

    y_proba = rf.predict_proba(X_test)[:, 1]
    roc_prob = roc_auc_score(y_test, y_proba)

    recall = recall_score(y_test, predictions)
    
    print(f"F1 score: {f1:.4f}, Recall Score: {recall:.4f} ROC_AUC score: {roc_prob:.4f}")
    return

In [32]:
print('Para df con valores nulos rellenados con 0:')
test_rf(X_train_no, y_train_no, X_test_no, y_test_no)
print()
print('Para df con valores nulos rellenados con 2:')
test_rf(X_train_alt, y_train_alt, X_test_alt, y_test_alt)
print()
print('Para df con features seleccionadas por Boruta:')
test_rf(X_train_bor, y_train_bor, X_test_bor, y_test_bor)

Para df con valores nulos rellenados con 0:
F1 score: 0.6161, Recall Score: 0.5408 ROC_AUC score: 0.8558

Para df con valores nulos rellenados con 2:
F1 score: 0.6201, Recall Score: 0.5429 ROC_AUC score: 0.8554

Para df con features seleccionadas por Boruta:
F1 score: 0.6159, Recall Score: 0.5730 ROC_AUC score: 0.8520


La diferencia entre usar cualquiera de los 3 dataframes es relativamente baja, pero el de los features determinados por Boruta nos dan el recall score más alto de los 3, así que podemos trabajar con este.

## Modelos de Machine Learning

Vamos a probar diferentes modelos para ver cuál es el que mejor nos funciona, hizimos una prueba preeliminar con Random Forest para elegir con qué características trabajar, pero podemos mejorarlo consdierablemente con ajuste de hiper parámetros. Los modelos que voy a probar son SelectKBest, RandomForest, LogisticRegression, DecisionTreeClassifier y LGMB Classifier.

### KNeighborsClassifier

In [33]:
X = df_bor.drop(columns='churn')
y = df_bor['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

#Realizamos el escalado de las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=123)

In [34]:
def evaluate_model(model, x_test, y_test, preds):
    f1 = f1_score(y_test, preds)
    y_proba = model.predict_proba(x_test)[:, 1]
    roc_prob = roc_auc_score(y_test, y_proba)
    recall = recall_score(y_test, preds)

    return {
        'F1': round(f1, 4),
        'Recall': round(recall, 4),
        'ROC AUC': round(roc_prob, 4)
    }

In [35]:
#Obtenemos el mejor valor de k para KNeighborsClassifier
k_range = range(1, 31)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    cv_scores = cross_val_score(knn, X_scaled, y, cv=5, scoring= None)
    scores.append(np.mean(cv_scores))

# Mostrar mejor k
best_k = k_range[np.argmax(scores)]
print(f"Mejor valor de k: {best_k}")
print(f"Score promedio con k={best_k}: {max(scores):.4f}")

Mejor valor de k: 22
Score promedio con k=22: 0.8258


In [36]:
#Y ahora creamos el modelo con el mejor k
knn = KNeighborsClassifier(n_neighbors=22)
knn.fit(X_train_scaled, y_train)
k_preds = knn.predict(X_test_scaled)

final_knn = evaluate_model(knn, X_test_scaled, y_test, k_preds)
final_knn

{'F1': 0.5859, 'Recall': 0.5088, 'ROC AUC': 0.8603}

Es muy bajo el recall score para KNN, vamos a ver si los otros modelos mejoran. Sus métricas fueron:

F1 score: 0.5859, Recall Score: 0.5088,  ROC_AUC score: 0.8603

### LogisticRegression

In [37]:
lr = LogisticRegression(max_iter=1000, random_state=123, solver='liblinear', class_weight='balanced')
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_test_scaled)
evaluate_model(lr, X_test_scaled, y_test, lr_preds)

{'F1': 0.5594, 'Recall': 0.7588, 'ROC AUC': 0.8107}

In [38]:
lr_1 = LogisticRegression(max_iter=1000, random_state=123, solver='liblinear', class_weight='balanced')
lr_1.fit(X_train, y_train)
lr_1_preds = lr_1.predict(X_test)
evaluate_model(lr_1, X_test, y_test, lr_1_preds)

{'F1': 0.5646, 'Recall': 0.7522, 'ROC AUC': 0.808}

In [39]:
lr_2 = LogisticRegression(max_iter=1000, random_state=123, solver='liblinear', class_weight='balanced')
lr_2.fit(X_train_no, y_train)
lr_2_preds = lr_2.predict(X_test_no)
evaluate_model(lr_2, X_test_no, y_test_no, lr_2_preds)

{'F1': 0.3943, 'Recall': 0.5687, 'ROC AUC': 0.5577}

In [40]:
#Regresión logística con el dataframe rellenado con 2
lr_3 = LogisticRegression(max_iter=1000, random_state=123, solver='liblinear', class_weight='balanced')
lr_3.fit(X_train_alt, y_train_alt)
lr_3_preds = lr_3.predict(X_test_alt)
final_lr = evaluate_model(lr_3, X_test_alt, y_test_alt, lr_3_preds)
final_lr

{'F1': 0.6068, 'Recall': 0.7833, 'ROC AUC': 0.8284}

El modelo de Regresión Logística que mejores resultados nos dio fue el que usa los datos nulos que se rellenaron con dos, muy cerca del que usó sólo las caracerísticas seleccionadas por boruta, pero con mejor evaluación en todas las métricas utilizadas.

### Random Forest

Ahora vamos a probar RandomForestClassifier, vamos a buscar primero los mejores parámetros para nuestro modelo, y después lo probaremos 

In [41]:
def best_rf(X_train, X_test, y_train = y_train, y_test = y_test):
    best_f1 = 0
    best_recall = 0
    best_est = 0
    best_depth = 0

    for est in range(1, 51):
        for depth in range(1, 21):
            model_rf = RandomForestClassifier(
                random_state=12345,
                n_estimators=est,
                max_depth=depth
            )
            model_rf.fit(X_train, y_train)
            rf_test_preds = model_rf.predict(X_test)

            f1 = f1_score(y_test, rf_test_preds)
            recall = recall_score(y_test, rf_test_preds)

            if f1 > best_f1:
                best_f1 = f1
                best_recall = recall
                best_est = est
                best_depth = depth

    print(f"El mejor modelo de Random Forest tiene {best_est} estimadores, Profundidad máxima de {best_depth}, F1-score: {best_f1:.4f}, y Recall: {best_recall:.4f}")
    return

In [42]:
#features con boruta
#best_rf(X_train, X_test)

Para features con boruta:

El mejor modelo de Random Forest tiene 24 estimadores, Profundidad máxima de 15, F1-score: 0.6201, y Recall: 0.5746

In [43]:
#features rellenados con 2
#best_rf(X_train_alt, X_test_alt)

Para features rellenados con 2:

El mejor modelo de Random Forest tiene 1 estimadores, Profundidad máxima de 20, F1-score: 0.3026, y Recall: 0.3026

In [44]:
#features rellenados con 0
#best_rf(X_train_no, X_test_no)

Para features rellenados con 0:

El mejor modelo de Random Forest tiene 1 estimadores, Profundidad máxima de 18, F1-score: 0.2892, y Recall: 0.2895

Lo que más conviene es utilizar el modelo con Boruta

In [45]:
rf = RandomForestClassifier(n_estimators=24, max_depth=15, random_state=12345)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
final_rf = evaluate_model(rf, X_test, y_test, rf_preds)
final_rf

{'F1': 0.6201, 'Recall': 0.5746, 'ROC AUC': 0.843}

### Árbol de decisión

In [46]:
best_tree_depth = 0
best_accuracy = 0
for depth in range(1,51): 
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(X_train, y_train)
    tree_preds = model_tree.predict(X_test)
    tree_score = accuracy_score(y_test, tree_preds)
    if tree_score > best_accuracy:
        best_tree_depth = depth
        best_accuracy= tree_score
print('Mejor profundidad =', best_tree_depth, 'con exactitud de', best_accuracy)

Mejor profundidad = 4 con exactitud de 0.8139931740614335


In [47]:
best_tree_depth = 0
best_accuracy = 0
for depth in range(1,51): 
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(X_train, y_train)
    tree_preds = model_tree.predict(X_test)
    tree_score = recall_score(tree_preds, y_test)
    if tree_score > best_accuracy:
        best_tree_depth = depth
        best_accuracy= tree_score
print('Mejor profundidad =', best_tree_depth, 'con recall de', best_accuracy)

c:\Users\Oliver\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mejor profundidad = 6 con recall de 0.714765100671141


Mejor profundidad = 4 con exactitud de 0.8139931740614335

In [48]:
tree = DecisionTreeClassifier(random_state=12345, max_depth=4)
tree.fit(X_train, y_train)
tree_preds = tree.predict(X_test)
decision_tree_final = evaluate_model(tree, X_test, y_test, tree_preds)
decision_tree_final

{'F1': 0.5948, 'Recall': 0.5263, 'ROC AUC': 0.8341}

In [49]:
tree = DecisionTreeClassifier(random_state=12345, max_depth=6)
tree.fit(X_train, y_train)
tree_preds = tree.predict(X_test)
evaluate_model(tree, X_test, y_test, tree_preds)

{'F1': 0.565, 'Recall': 0.4671, 'ROC AUC': 0.8542}

### LGMBClassifier

In [50]:


best_score = 0
best_est= 0
best_depth = 0

for est in range(1, 301, 25):
    for depth in range(1, 31):
        model_lgbm = LGBMClassifier(n_estimators=est, max_depth=depth, random_state=12345,
                                    verbose = -1) #Para evitar que sature el notebook con logs.
        model_lgbm.fit(X_train, y_train)

        lgbm_preds = model_lgbm.predict(X_test)
        score = recall_score(y_test, lgbm_preds)
        if score > best_score:
            best_score = score
            best_est= est
            best_depth = depth

print('El mejor modelo de LGBM es con', best_est, 'estimadores, una profundidad máxima de', best_depth, 'y da un recall score de', best_score)

El mejor modelo de LGBM es con 176 estimadores, una profundidad máxima de 20 y da un recall score de 0.5614035087719298


El mejor modelo de LGBM para df_bor es con 101 estimadores, una profundidad máxima de 10 y da un f1 score de 0.6036363636363636



El mejor modelo de LGBM para df_alt es con con 176 estimadores, una profundidad máxima de 8 y da un f1 score de 0.07879924953095685

El mejor modelo de LGBM para df_no es con con 176 estimadores, una profundidad máxima de 9 y da un f1 score de 0.09157509157509157


Se realizaron más pruebas, pero con diferentes parámetros no mejoran significativamente, la mejor opción para LGBM es utilizar los features propuestos por boruta.

In [51]:
lgbm = LGBMClassifier(n_estimators=101, max_depth = 10, random_state=12345, verbose=-1)
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(X_test)
final_lgbm =evaluate_model(lgbm, X_test, y_test, lgbm_preds)
final_lgbm

{'F1': 0.6036, 'Recall': 0.5461, 'ROC AUC': 0.8555}

### XGBoost

In [52]:
def rate_xgb(X_train, X_test, y_train= y_train, y_test=y_test):
    best_recall = 0
    best_estimators = 0
    best_depth = 0

    for est in range(10, 201, 10):
        for depth in range(3, 15):
            model = XGBClassifier(
                n_estimators=est,
                max_depth=depth,
                eval_metric='logloss',
                random_state=42
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            recall = recall_score(y_test, y_pred)

            if recall > best_recall:
                best_recall = recall
                best_estimators = est
                best_depth = depth

    print('El mejor modelo de XGBoost es con', best_estimators, 'estimadores, una profundidad máxima de', best_depth, 'y da un score de', best_recall)
    return

In [53]:
#rate_xgb(X_train_alt, X_test_alt)

El mejor modelo de XGBoost es con 50 estimadores, una profundidad máxima de 13 y da un recall score de 0.5855263157894737

El mejor modelo de XGBoost es con 10 estimadores, una profundidad máxima de 12 y da un f1 score de 0.6173708920187794



Los mejores resultados para XGBoost también fueron utilizando la versión del datarame de features seleccionados por Boruta

In [54]:
xgb = XGBClassifier(n_estimators=50, max_depth=13, random_state=12345)
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)
final_xgb = evaluate_model(xgb, X_test, y_test, xgb_preds)
final_xgb

{'F1': 0.6152, 'Recall': 0.5855, 'ROC AUC': 0.8463}

## Selección del modelo

Ya hicimos la prueba de varios modelos, vamos a hacer una comparación del rendimiento de cada uno para ver cuál elegimos

In [55]:
results = pd.DataFrame([
    {'Modelo': 'KNN', **final_knn},
    {'Modelo': 'LogisticRegression', **final_lr},
    {'Modelo': 'RandomForest', **final_rf},
    {'Modelo': 'DecisionTree', **decision_tree_final},
    {'Modelo': 'LGBM', **final_lgbm}, 
    {'Modelo': 'XGB', **final_xgb}
])

print(results.set_index('Modelo'))


                        F1  Recall  ROC AUC
Modelo                                     
KNN                 0.5859  0.5088   0.8603
LogisticRegression  0.6068  0.7833   0.8284
RandomForest        0.6201  0.5746   0.8430
DecisionTree        0.5948  0.5263   0.8341
LGBM                0.6036  0.5461   0.8555
XGB                 0.6152  0.5855   0.8463


El modelo que mejor se desempeña para nuestro objetivo es LogisticRegression, tiene un valor F1 de .60 y un recall score de .78, con un valor de ROC_AUC ligeramente menor al resto, pero en nuestra métrica objetivo de recall se desempeña considerablemente mejor. Éste se trabajó con el dataframe que usaba todos los features, y rellenaba los datos ausentes con un 2.

In [56]:
final_model = lr_3
X = df_alt.drop(columns= ['customer_id', 'churn'])

Con nuestro modelo, podemos utilizar la función que tiene para predecir probabilidad (.predict_proba) y utilizar los datos de personas que no han cancelado y tengan alta probabilidad de cancelación, dependiendod el umbral que se establezca, para ofrecer promociones u otro tipo de atención personalizada.

In [57]:
df['churn_prob'] = final_model.predict_proba(X)[:,1]
df.sample(10)

,customer_id,churn,payment_type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,begin_year,churn_prob
5590,7872-RDDLZ,0,0,0,0,54.90,3725.50,1.0,0.0,1.0,...,0.0,0.0,0.0,1,1,0,0,1.0,2014,0.417433
2230,3190-XFANI,1,1,1,2,100.60,5069.65,2.0,0.0,0.0,...,1.0,1.0,1.0,0,0,0,1,1.0,2015,0.372419
1859,2696-ECXKC,0,1,0,1,100.90,5448.60,2.0,1.0,0.0,...,0.0,1.0,1.0,1,0,1,1,1.0,2015,0.288006
4958,6980-IMXXE,0,2,0,2,20.20,1412.65,0.0,NaN,NaN,...,NaN,NaN,NaN,1,0,1,1,0.0,2014,0.030367
5903,8313-AFGBW,0,2,0,0,73.60,3522.65,1.0,0.0,1.0,...,0.0,1.0,1.0,0,0,1,0,0.0,2016,0.119776
836,1217-VASWC,0,1,1,2,100.55,4304.00,2.0,0.0,1.0,...,0.0,1.0,1.0,0,1,1,0,0.0,2016,0.480688
6549,9300-AGZNL,1,0,1,0,94.00,94.00,2.0,0.0,0.0,...,0.0,1.0,1.0,0,1,0,0,1.0,2019,0.941352
2744,3902-MIVLE,0,2,0,1,75.70,4676.70,1.0,1.0,0.0,...,1.0,0.0,1.0,0,0,1,1,1.0,2014,0.037591
3287,4695-VADHF,1,0,0,0,57.45,990.85,1.0,0.0,0.0,...,0.0,0.0,1.0,0,0,1,1,0.0,2018,0.541687
6472,9167-APMXZ,0,0,1,2,84.15,1821.95,2.0,0.0,0.0,...,0.0,1.0,0.0,1,0,0,0,1.0,2018,0.806325


Logramos predecir con éxito la probabilidad de que un cliente vaya a cancelar, lo que podemos hacer ahora es una lista de los clientes que es probable que cancelen para que la empresa los contacte para dar seguimiento al caso antes que tomen alguna decisión desfavorable.

El umbral está considerado como .5, si la empresa sólo se quiere enfocar en únicamente los clientes más urgentes, puede moficiar este umbral.

## Poducto final

In [58]:
tolerance = .5
df_to_contact = df[(df['churn_prob'] > tolerance) & (df['churn'] == 0)][['customer_id', 'churn_prob']]
df_to_contact.sort_values(by='churn_prob', ascending=False)

,customer_id,churn_prob
3641,5150-ITWWB,0.937674
4504,6350-XFYGW,0.927149
3421,4847-TAJYI,0.924549
2887,4115-NZRKS,0.924418
2323,3320-VEOYC,0.921200
...,...,...
4212,5948-UJZLF,0.500856
2558,3657-COGMW,0.500232
2451,3511-APPBJ,0.500211
2501,3577-AMVUX,0.500177


## Conclusión

Logramos hacer un modelo que predice la probabilidad de que un cliente se vaya a ir por medio de la Regresión Logística. La métrica en la que nos enfocamos fue recall_score ya que nos ayuda con los verdaderos positivos, minimizando así que no detecte a un cliente que se iba a ir y el modelo no lo detectó, y este modelo, además de tener otras métricas comparables a las de los otros modelos que se probaron, tenía un desempeño considerablemente mejor con el recall_score.

Ahora tenemos la posibilidad de utilizar este modelo, ajustando el margen de tolerancia si se desea, para encontrar a los clientes que es probable que vayan a cancelar y así poder tomar acciones con ellos en específico, ahorrando recursos a la empresa y reduciendo la tasa de clientes que cancelan el servico.

## Informe de solución

Se había proupesto un plan inicial de trabajo antes de empezar a hacer lo que necesitábamos. Se plantearon unas preguntas a responder, que quedaron parcialmente contestadas, la primera siendo ¿Qué factores influyen más a que la gente cancele el servicio? No se contestó explícitamente, pero la selección de características con Boruta, que sugirió el equipo, nos devolvió las que tienen más peso en la tasa de cancelación; estas son el método de pago, los cargos mensuales y totales, y el año de inicio, lo que va de la mano con la tercera pregunta, que pregutnaba qué tanto influye si tienen internet o teléfono contratado, y la respuesta va relacionada a los cargos mensuales, si tienen los dos servicios, la cuenta es mayor. La segunda pregunta, de si podemos crear un modelo que pueda predecir precisamente los clientes que cancelan, es el resultado final del trabajo, aunque el modelo creado no es completamente preciso, genera la la probabilidad de que un cliente vaya a cancelar, que es justo lo que necesitamos y le logró exitosamente.

Del plan de trabajo original, se llevó a cabo la base de lo propuesto, que fue unificar los datasets, convertir columnas categóricas a numéricas, pero el resto fue diferente. El equipo me comentó acertadamente que la métrica f1 podría no ser la más adecuada para el caso, así que opté por recall score, sin descuidar el resto de métricas para que no se enfoque únicamente en recall, porque los modelos que tenían un recall excepcional, fallaban considerablemente en las demás métricas.

Por último, se había propuesto basarnos en los pesos que tenía nuestro modelo para cada columna, pero fue una mejor opción utilizar la función de predecir probabilidad, y trabajar con esa, ya que le da al equipo de retención los datos de a quién se le tiene que contactar y les da la herramienta de priorizar a los que más alta probabilidad tienen de cancelar.

Una de las principales dificultades fue saber qué hacer con los valores ausentes, para esto analicé muchas opciones y opté por no asumir que simplemente significaban que el cliente no tenía el servicio, podía ser un error del sistema, así que trabajé los valores ausentes como un dato diferente a un sí y no, sin descartar la posibilidad de que fueran no, y seguí haciendo las pruebas con todas mis opciones para aseguar que los diferentes modelos pudieran mejorar dependiendo de la calidad de información que le brindaba cada dataframe. También tuve ciertas complicaciones con el código para escoger los mejor hiperparámetros de cada modelo, ya que al probar diferentes métricas, a veces no eran compatibles con el código que ya tenía y tenía que buscar cómo adaptarlo para que me diera la información que necesitaba.

Los pasos clave apra resolver la tarea de forma satisfactoria fueron determinar la métrica que más aplicaba al problema, el recall score fue un aspecto primordial para determinar el mejor modelo, pero era muy importante también poner atención a las demás métricas que estábamos trabajando, por ejemplo random forest sugería que lo que maximizaba el recall era utilizar sólo un estimador, pero eso es desaprovechar completamente el modelo. Fuera de eso, sólo tuve problemas con que todos los dataframes fueran igualmente trabajables, ya que ciertos códigos no eran complatibles y tenía que hacer modificiaciones específicas, tuve que hacer mucho backtracking pero al final todo quedó en orden.

El modelo que se eligió fue una Regresión Logística, ya que tenía métricas de f1 y ROC_AUC competitivas con el resto de modelos que se probaron, pero superaba considerablemente al resto en recall score, que es la que nos aseguraba que era más probable que se detecte a un cliente que podía cancelar. Sus métricas fueron: F1': 0.6068, 'Recall': 0.7833, 'ROC AUC': 0.8284 